# Backtesting and report

In [1]:
import os
import pandas as pd
from orats_backtest_v12 import Backtester
from backtest_report import option_trade_report
import logging

In [2]:

max_dte = 8 # 8 days to expiration
min_rr = 0.8 # min risk/reward ratio
max_spread_pct = 0.15 # max spread percentage of underlying price
deltas = [round(i / 100, 2) for i in range(10, 101, 5)] # delta range for options to be traded
print('max_dte:', max_dte)
print('min_rr:', min_rr)
print('max_spread_pct:', max_spread_pct)
print('deltas:', deltas)

max_dte: 8
min_rr: 0.8
max_spread_pct: 0.15
deltas: [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]


In [3]:
# all the signal files are in the same directory, they are named part_001.csv, part_002.csv, etc.
signal_file_ids = [
    '001', '002',
    '003', '004', '005', '006', '007', '008', '009', '010', 
    '011', '012', '013', '014', '015', '016', '017', '018', '019', '020'
    ]

# run the backtest for each signal file
for signal_file_id in signal_file_ids:
    bt = Backtester(
        signals_path=f"signal-files/part_{signal_file_id}.csv",
        orats_path=f"output/part_{signal_file_id}_entry.csv.gz", # orats entry file
        exit_path="output/exit.csv.gz", # orats exit file
        deltas=deltas,
        max_spread_pct=max_spread_pct,
        min_rr=min_rr,
        max_dte=max_dte
    )

    trades = bt.run()
    if not trades.empty:
        os.makedirs("results", exist_ok=True)
        out_file = f"results/part_{signal_file_id}_trades.csv"
        trades.to_csv(out_file, index=False)
        logging.info("Trade log saved to %s", out_file)


2025-05-22 21:23:41,847 INFO: Loading CSVs …
2025-05-22 21:24:33,661 INFO: Trade log saved to results/part_001_trades.csv
2025-05-22 21:24:33,662 INFO: Loading CSVs …
2025-05-22 21:25:37,597 INFO: Trade log saved to results/part_002_trades.csv
2025-05-22 21:25:37,598 INFO: Loading CSVs …
2025-05-22 21:26:38,732 INFO: Trade log saved to results/part_003_trades.csv
2025-05-22 21:26:38,733 INFO: Loading CSVs …
2025-05-22 21:27:38,448 INFO: Trade log saved to results/part_004_trades.csv
2025-05-22 21:27:38,450 INFO: Loading CSVs …
2025-05-22 21:28:36,727 INFO: Trade log saved to results/part_005_trades.csv
2025-05-22 21:28:36,728 INFO: Loading CSVs …
2025-05-22 21:29:41,164 INFO: Trade log saved to results/part_006_trades.csv
2025-05-22 21:29:41,165 INFO: Loading CSVs …
2025-05-22 21:30:51,374 INFO: Trade log saved to results/part_007_trades.csv
2025-05-22 21:30:51,375 INFO: Loading CSVs …
2025-05-22 21:32:00,170 INFO: Trade log saved to results/part_008_trades.csv
2025-05-22 21:32:00,170 

In [5]:
# joint all trades into one dataframe
trades = pd.DataFrame()
for filename in os.listdir('results'):
    if filename.endswith('_trades.csv'):
        trades = pd.concat([trades, pd.read_csv(os.path.join('results', filename))], ignore_index=True)
        
# save the combined trades to a single file
trades.to_csv('results/all_trades.csv', index=False)

print(trades.head())

                   timestamp  signal_id              name  \
0  2023-05-03 15:34:53+00:00          3   call_spread_0.3   
1  2023-05-03 15:34:53+00:00          3   call_spread_0.3   
2  2023-05-05 20:15:00+00:00          3   call_spread_0.3   
3  2023-05-05 20:15:00+00:00          3   call_spread_0.3   
4  2023-05-03 15:34:53+00:00          3  call_spread_0.35   

              order_ref  quantity          order symbol asset right  \
0   3|call_spread_0.3|1         1    Buy-to-Open    MPC   OPT  Call   
1   3|call_spread_0.3|1        -1   Sell-to-Open    MPC   OPT  Call   
2   3|call_spread_0.3|2        -1  Sell-to-Close    MPC   OPT  Call   
3   3|call_spread_0.3|2         1   Buy-to-Close    MPC   OPT  Call   
4  3|call_spread_0.35|1         1    Buy-to-Open    MPC   OPT  Call   

   multiplier  ... filled_price  credit  transaction_fee  underlier     delta  \
0         100  ...         0.77   -77.0              0.5     111.73  0.294956   
1         100  ...         0.27    27.0     